# PURPOSE OF THIS NOTEBOOK
This is the base code to write the functions to extract the raw text from resumes and append to our dataframe. 

TODO: Take the functions into a .py folder and use it as a script 

In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from tokenizers import ByteLevelBPETokenizer
from transformers import BertTokenizer, BertModel
from pathlib import Path
from torch import cuda
import torch

from sklearn import metrics
import transformers
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

C:\Users\FrostyBB\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Setting up for GPU 

device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
device

'cuda'

## Model Goal

The goal is to train a custom BERT model to attempt to label each incoming job title to their proper ONET code. ONET (Occupational Information Network) is a free database that serves as a standardized taxonomy for jobs. Each job has a respective standardized name and code associated with it. This will make extracting skills quite easy if we can corrrectly translate job titles to the proper ONET code. Using BERT, we can tokenize the job titles and match them with the database of common job titles for each ONET code. For this first version I will be leaving out the actual ONET job names from the training data to compare later with an updated dataset. More information about ONET can be found here: https://www.onetonline.org/

In [4]:
# Import the train/test Data. 
test_df = pd.read_csv("../Data/TestingData.csv")
train_df = pd.read_csv("../Data/Training_Data.csv")
label_df = pd.read_csv("../Data/label_df.csv")

In [5]:
# Check that the test data incoming is correct
test_df.head()

,Reported_Jobs,Label
0,Executive Vice President (EVP),"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Arts and Humanities Council Director,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Bank President,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Bureau Chief,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Business Development Executive,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [6]:
# Check that the training data incoming is correct 
train_df.head()

,Reported_Jobs,Label
0,Safety Council Director,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Chief Financial Officer (CFO),"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Council On Aging Director,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Operations Vice President (Operations VP),"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Regulatory Agency Director,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [7]:
train_df['Label'] = train_df['Label'].apply(lambda s: [float(x.strip(' []')) for x in s.split(',')])

In [8]:
test_df['Label'] = test_df['Label'].apply(lambda s: [float(x.strip(' []')) for x in s.split(',')])

## The goal of the tokenizer

The tokenizer will be tokenizing the job titles and the reported job titles, I don't know if I should do these seperately or together. In theory, I should have these pairings be tokenized together. **I SHOULD READ INTO THE TOKENIZER TO UNDERSTAND HOW I SHOULD APPROACH THIS** 

From looking at the reference code, I've learned that we need to follow these steps: 
1. Start with a train test split. **70% for the training data**, I will do the split based on the **70%** of each reported job title/ONET pairing.
    - Given the refernece notebook uses a dictionary as the input data and we are working with a dataframe instead, some major changes will be needed to be made in order for this to work. I don't think this would be difficult at all. Just need to translate the dictionary work to the dataframe. **I also need to confirm if the model input requires a list, dict, or dataframe object.**
2. Run the tokenizer on the training set 
3. Set up the model training and evaluation metrics.


In [9]:
# Load in the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [10]:
MAX_LEN = 175
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 20
LEARNING_RATE = 2e-5

In [11]:
# Start by tokenizing the data.
# Will be using the class statment and slowly converting it for our needs 
class CustomDataset(Dataset):
    
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.reported_jobs = dataframe.Reported_Jobs
        self.targets = self.data.Label
        self.max_len = max_len
    
    def __len__(self):
        return len(self.reported_jobs)
    
    def __getitem__(self, index):
        # This can be done with a for loop 
        reported_job = str(self.reported_jobs[index])
        reported_job = " ".join(reported_job.split())

        inputs = self.tokenizer.encode_plus(
            reported_job,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs['token_type_ids']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [12]:
def predict(input, tokenizer, model, device):
    ''' GOAL OF THIS FUNCTION: 
    This function takes in any given string and converts it into a tokenized version that can be run through the model. '''

    input = " ".join(input.split())
    inputs = tokenizer.encode_plus(
        input,
        None,
        add_special_tokens=True,
        max_length=50,
        pad_to_max_length=True,
        return_token_type_ids=True
    )

    ids = inputs['input_ids']
    mask = inputs['attention_mask']
    token_type_ids = inputs['token_type_ids']

    ids = torch.tensor(ids, dtype=torch.long)
    mask = torch.tensor(mask, dtype=torch.long)
    token_type_ids = torch.tensor(mask, dtype=torch.long)


    output = model(ids.unsqueeze(0), mask.unsqueeze(0), token_type_ids.unsqueeze(0))

    return output

In [13]:
# Running the tokenizer and shaping the dataframes for the model
train_set = CustomDataset(train_df, tokenizer, MAX_LEN)
test_set = CustomDataset(test_df, tokenizer, MAX_LEN)

In [14]:
# Setting the train and test parameters 
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0}

test_params = {'batch_size': VALID_BATCH_SIZE,
               'shuffle': True,
               'num_workers': 0}

training_loader = DataLoader(train_set, **train_params)
testing_loader = DataLoader(test_set, **test_params)

In [15]:
# Creatring the custom model

class BERTClass(torch.nn.Module):
    def __init__(self):
        # Defining the layers
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased', return_dict=False)
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 997)
    
    def forward(self, ids, mask, token_type_ids):
        _, output_1 = self.l1(ids, attention_mask=mask, token_type_ids=token_type_ids)
        output_2 = self.l2(output_1)
        output_3 = self.l3(output_2)
        return output_3

model = BERTClass()
model.to(device)

BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [16]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)

# Fine tuning

In [17]:
def train(epoch):
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss= loss_fn(outputs, targets)
        if _%100==0:
            print(f'Epoch : {epoch}, Loss: {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    return outputs

In [18]:
for epoch in range(EPOCHS):
    output = train(epoch)

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


C:\Users\FrostyBB\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
3it [00:01,  2.16it/s]

Epoch : 0, Loss: 0.7093998193740845


103it [00:09, 13.32it/s]

Epoch : 0, Loss: 0.2521761655807495


...

5302it [06:48, 14.40it/s]

Epoch : 19, Loss: 0.001681604771874845


5402it [06:55, 13.38it/s]

Epoch : 19, Loss: 0.001044943230226636


5481it [07:01, 13.00it/s]


## Model Evaluation and testing 

In [19]:
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _,data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            
            outputs = model(ids, mask, token_type_ids)

            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [20]:
for epoch in range(EPOCHS):
    outputs, targets = validation(epoch)
    outputs = np.array(outputs) >= 0.5
    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")

4708it [01:05, 72.14it/s]


Accuracy Score = 0.3692176961070689
F1 Score (Micro) = 0.4516108247422681
F1 Score (Macro) = 0.42218819187851525


0it [00:00, ?it/s]C:\Users\FrostyBB\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
4708it [01:06, 71.23it/s]


Accuracy Score = 0.3692176961070689
F1 Score (Micro) = 0.4516108247422681
F1 Score (Macro) = 0.42218819187851525


0it [00:00, ?it/s]C:\Users\FrostyBB\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
521it [00:07, 70.45it/s]


KeyboardInterrupt: 

## Test Predictions with the model 

In [22]:
# To predict, the model is looking for the ids, mask and token_type_ids. Which means whatever we input must be run through the tokenizer first. 
input='Executive Director'
output = predict(input, tokenizer, model, device)

C:\Users\FrostyBB\AppData\Local\Temp\ipykernel_14132\4155948468.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  token_type_ids = torch.tensor(mask, dtype=torch.long)


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [23]:
output_df = pd.DataFrame(output.cpu().detach().numpy().tolist(), columns=label_df.columns)

In [24]:
output_df

,11-1011.00,11-1011.03,11-1021.00,11-1031.00,11-2011.00,11-2021.00,11-2022.00,11-2032.00,11-2033.00,11-3012.00,...,53-7062.04,53-7063.00,53-7064.00,53-7065.00,53-7071.00,53-7072.00,53-7073.00,53-7081.00,53-7121.00,53-7199.00
0,-12.677019,-11.923305,-12.271404,-13.911635,-11.803603,-12.689295,-11.907290,-10.829463,-10.991726,-13.121556,...,-13.236665,-10.214296,-12.002809,-10.842522,-11.167063,-10.496148,-9.829696,-13.442833,-13.831484,-12.543354
1,-9.519832,-10.484120,-9.515649,-12.371419,-11.096474,-9.485592,-10.091171,-10.393816,-10.612860,-11.368751,...,-4.924407,-4.599410,-3.124410,-2.436879,-7.743529,-5.805891,-6.982162,-3.806403,-5.551823,-3.583666


In [25]:
output_df.idxmax(axis='columns')

0    29-2053.00
1    53-7062.00
dtype: object

In [ ]:
output_df.idxmin(axis=1)